In [1]:
import sys
sys.path.append( '/anaconda/envs/py37_pytorch/lib/python3.7')
sys.path.append('/anaconda/envs/py37_pytorch/lib/python3.7/site-packages')
%load_ext autoreload
%autoreload 2

import glob
import logging
import numpy as np
from importlib import reload  # Not needed in Python 2
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.models import ResNet
from torch.utils.data import DataLoader,Dataset


from dataset.ASdataset import AS_Data
from dataset.ASdataset_obs_train_input import AS_Data_obs

device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
reload(logging)
logging.basicConfig(level=logging.INFO,#控制台打印的日志级别
                    filename='logging.txt',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format=
                    '%(asctime)s : %(message)s',
                    )

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
from copy import deepcopy

In [2]:
voc = [0,2,3,4,8,9,10] + list(range(32,51))
pm25 = list(range(10,30))
rest = [1,5,30,31]
no = [6,7]

In [3]:
for filename in sorted(glob.glob('/AS_data/Emis_with_foreign_npy/EM_2015_10_744_51_182_232.npy')):
    print(filename)
    org_data = np.load(filename)
    print(org_data.shape)
for filename in sorted(glob.glob('/AS_data/zeyuan_folder/concat_data/rest_EM_2015_10_744_07_182_232.npy')):
    print(filename)
    con_data = np.load(filename)
    print(con_data.shape)
for filename in sorted(glob.glob('/AS_data/finetune_input_prelu_VOCsame_PM25_O3_NO2_SO2/rest_4month_SULF_CO_7dim_10_finetune_input.npy')):
    print(filename)
    fin_data = np.load(filename)
    print(fin_data.shape)

/AS_data/Emis_with_foreign_npy/EM_2015_10_744_51_182_232.npy
(744, 51, 182, 232)
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_10_744_07_182_232.npy
(744, 7, 182, 232)
/AS_data/finetune_input_prelu_VOCsame_PM25_O3_NO2_SO2/rest_4month_SULF_CO_7dim_10_finetune_input.npy
(744, 7, 182, 232)


In [4]:
final_data = deepcopy(org_data)
delta_data = fin_data - con_data
print(final_data.shape)
print(delta_data.shape)

(744, 51, 182, 232)
(744, 7, 182, 232)


In [5]:
print('Updating VOC')
delta_voc = delta_data[:,4,:,:]
mean_voc = con_data[:,4,:,:]
for i in voc:
    final_data[:,i,:,:] += (org_data[:,i,:,:] + 1e-5) * delta_voc / (mean_voc + 1e-5)
print('Updating PM25')
delta_voc = delta_data[:,5,:,:]
mean_voc = con_data[:,5,:,:]
for i in pm25:
    final_data[:,i,:,:] += (org_data[:,i,:,:] + 1e-5) * delta_voc / (mean_voc + 1e-5)
print('Updating NO')
delta_voc = delta_data[:,6,:,:]
mean_voc = con_data[:,6,:,:]
for i in no:
    final_data[:,i,:,:] += (org_data[:,i,:,:] + 1e-5) * delta_voc / (mean_voc + 1e-5)

Updating VOC
Updating PM25
Updating NO


In [6]:
with open('/AS_data/zeyuan_folder/reload_data/adjust_7EM_10_744_51_182_232.npy','wb') as f:
    np.save(f,final_data)